In [1]:
import tensorflow as tf
tf.enable_eager_execution()
import numpy as np

### tf.where(condition, x, y, name)
- 作用: 返回condition为True的元素坐标(x=y=None)
- 返回维度: (num_true, dim_size(condition))
    - 其中dim_size为condition的维度

In [2]:
ind = np.array([[[1, 0, 0], [0, 0, 2]],
         [[0, 0, 3], [0, 0, 0]],
         [[0, 5, 0], [6, 0, 0]]])
print(ind.shape)

(3, 2, 3)


In [3]:
tf.where(ind)

<tf.Tensor: id=2, shape=(5, 3), dtype=int64, numpy=
array([[0, 0, 0],
       [0, 1, 2],
       [1, 0, 2],
       [2, 0, 1],
       [2, 1, 0]])>

- 上述tf.where操作后返回的矩阵维度是(5,3)
- 其中5是指ind矩阵中有5个元素不为0(即True)
- 返回的[[0, 0, 0],[0, 1, 2],[1, 0, 2],[2, 0, 1],[2, 1, 0]]为对应元素的坐标
- 其中3是指ind矩阵的维度为3

##########################################################################

### tf.gather(params, indices, ..., axis=0)
- 作用: 根据indices从params的指定轴axis索引元素(类似于仅能在指定轴进行一维索引)
- 返回维度: params.shape[:axis] + indices.shape + params.shape[axis + 1:]

In [4]:
data = np.array([[[1, 1, 1], [2, 2, 2]],
         [[3, 3, 3], [4, 4, 4]],
         [[5, 5, 5], [6, 6, 6]]])
print(data.shape)

(3, 2, 3)


In [5]:
indices = np.array([0,2])
print(indices.shape)

(2,)


#### example 1

In [6]:
tf.gather(data, indices)

<tf.Tensor: id=7, shape=(2, 2, 3), dtype=int64, numpy=
array([[[1, 1, 1],
        [2, 2, 2]],

       [[5, 5, 5],
        [6, 6, 6]]])>

- axis=0, indices选择0,2, data为(3,2,3)
- 索引出的数组为[[[1, 1, 1],[2, 2, 2]], [[5, 5, 5],[6, 6, 6]]]是data矩阵axis=0轴的第1和第3个数组
- data.shape[:0]=(); indeices.shape=(2,); data.shpe[1:]=(2,3)
- 最后的输出大小为上述3个之和， () + (2,) + (2,3) => (2, 2, 3)

#### example 2

In [7]:
tf.gather(data, indices, axis=2)

<tf.Tensor: id=12, shape=(3, 2, 2), dtype=int64, numpy=
array([[[1, 1],
        [2, 2]],

       [[3, 3],
        [4, 4]],

       [[5, 5],
        [6, 6]]])>

- axis=2, indices选择[0,2], data为(3,2,3)
- 索引出的数组为[[[1, 1],[2, 2]], [[3, 3],[4, 4]], [[5, 5],[6, 6]]]是data矩阵axis=2轴的第1和第3个数组
- data.shape[:2]=(3,2); indeices.shape=(2,); data.shpe[2+1:]=()
- 最后的输出大小为上述3个之和， (3,2) + (2,) + (0) => (3, 2, 2)

### tf.gather_nd(params, indices, name=None)
- 作用:将params索引为indices指定形状的切片数组中(indices代表索引后的数组形状)
    - indices将切片定义为params的前N个维度，其中N = indices.shape [-1]
    - 通常要求indices.shape[-1] <= params.rank(可以用np.ndim(params)查看)
        - 如果等号成立是在索引具体元素
        - 如果等号不成立是在沿params的indices.shape[-1]轴进行切片
- 返回维度: indices.shape[:-1] + params.shape[indices.shape[-1]:]
    - 前面的indices.shape[:-1]代表索引后的指定形状
    
########################################################################

In [8]:
data = np.array([[[1, 1, 1], [2, 2, 2]],
         [[3, 3, 3], [4, 4, 4]],
         [[5, 5, 5], [6, 6, 6]]])
print("data is \n", data)
print("data shape is ",data.shape)
print("data rank is ", np.ndim(data))

data is 
 [[[1 1 1]
  [2 2 2]]

 [[3 3 3]
  [4 4 4]]

 [[5 5 5]
  [6 6 6]]]
data shape is  (3, 2, 3)
data rank is  3


#### example 1

In [9]:
indices = np.array([[0, 1], [1, 0]])
print(indices.shape)

(2, 2)


In [10]:
# (indices.shape[-1]=2) < (np.ndim(data)=3), 沿data的axis=2轴切片
tf.gather_nd(data, indices)

<tf.Tensor: id=16, shape=(2, 3), dtype=int64, numpy=
array([[2, 2, 2],
       [3, 3, 3]])>

- indices选择[[0, 1], [1, 0]], data为(3,2,3)
- [0,1]的索引过程为选取axis=0的第1个数组得到[[1,1,1],[2,2,2]],
 再选取axis=1的第2个数组得到[2,2,2]，[1,0]过程同理，最后索引出的是[[2,2,2],[3,3,3]]
- indices.shape[:-1]=(2,); indices.shape[-1]=2; 
    data.shpe[indices.shape[-1]:]=>data.shape[2:]=(3,)
- 最后的输出大小为上述2个之和， (2,) + (3,) => (2, 3)

#### example 2

In [11]:
indices = np.array([[[0, 0, 1], [1, 1, 0]]])
print(indices.shape)

(1, 2, 3)


In [12]:
# (indices.shape[-1]=3) < (np.ndim(data)=3), 取data对应位置的元素
tf.gather_nd(data, indices)

<tf.Tensor: id=20, shape=(1, 2), dtype=int64, numpy=array([[1, 4]])>

- indices选择[[[0, 0, 1], [1, 1, 0]]], data为(3,2,3)
- [0,0,1]的索引过程为选取axis=0的第1个数组得到[[1,1,1],[2,2,2]],
 再选取axis=1的第1个数组得到[1,1,1]，再取axis=3的第3个数组得到1,[1,1,0]过程同理，最后索引出的是[[1,4]]
- indices.shape[:-1]=(1,2); indices.shape[-1]=3; 
    data.shpe[indices.shape[-1]:]=>data.shape[3:]=()
- 最后的输出大小为上述2个之和， (1,2) + () => (1,2)